In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

In [2]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [3]:
#Surface data
t2mData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/tasNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')
rhData  = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/hursNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')
u10Data = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/uasNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')
v10Data = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/vasNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')

In [4]:
# Level Data
tLevData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/taNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')
zLevData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/zgNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')
wLevData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/wapNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')
uLevData = xr.open_dataset('../../../Data/CMIP6/GFDL-ESM4/uaNHRegrid_NDJFday_GFDL-CM4_historical_r1i1p1f1_gr2_18500101-20141231.nc')

In [5]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
rhD  = rhData.sel(latitude=slice(32,24),longitude=slice(74,86))
tLevD = tLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
uLevD = uLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = wLevData.sel(latitude=slice(32,24),longitude=slice(74,86)).sel(plev=70000,method='nearest')
zLevD = zLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
#w700D

In [6]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10D.vas.values**2)+(u10D.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhD.hurs)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

#Calculate inv
inv=t2mD.tas.values-tLevD.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, 
                    coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevD.ua.sel(plev=85000,method='nearest').values-u10D.uas.values)/(zLevD.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, 
                       coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [7]:
AOData = xr.open_dataset('GFDL-CM4-AOindex-NDJF-Daily-1980-2014.nc')

AO=AOData.AO
datetimeindex = AO.indexes['time'].to_datetimeindex()
datetimeindex
AO['time'] = datetimeindex
AO.values

AO5D=AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

<ipython-input-7-db568de8984f>:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = AO.indexes['time'].to_datetimeindex()


In [8]:
EUData = xr.open_dataset('GFDL-CM4-EUindex-NDJF-Daily-1980-2014.nc')
EU=EUData.EU
datetimeindex = EU.indexes['time'].to_datetimeindex()
datetimeindex
EU['time'] = datetimeindex
EU.values
EU5D=EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))


<ipython-input-8-268572d48fd1>:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = EU.indexes['time'].to_datetimeindex()


In [9]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [10]:
# Apply Mask 
t2mD.tas.values=t2mD.tas.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.wap.values=w700D.wap.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask

In [11]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).tas
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).wap
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear

In [12]:
datetimeindex = t2mTS1.indexes['time'].to_datetimeindex()
t2mTS1['time'] = datetimeindex
wsTS1['time'] = datetimeindex
rhTS1['time'] = datetimeindex
wTS1['time'] = datetimeindex
invTS1['time'] = datetimeindex
ushearTS1['time'] = datetimeindex

<ipython-input-12-cc712bdc98ec>:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t2mTS1.indexes['time'].to_datetimeindex()


In [13]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)

In [14]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

In [15]:
X       = pd.DataFrame([t2m.values,ws.values,rh.values/100.0,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
                        EU5DAll.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D']).T[:]
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,t2m,ws,rh,inv,w,ushear,AO5D,EU5D
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,283.896439,1.629439,0.579713,2.271117,0.022779,0.136908,-0.036220,0.019196
std,2.225390,0.504814,0.137624,1.978540,0.100687,0.118351,0.917866,0.593310
min,276.511017,0.453322,0.184784,-3.945501,-0.432205,-0.346720,-2.670958,-1.726749
25%,282.297844,1.260378,0.480503,0.850053,-0.032971,0.053105,-0.746139,-0.390820
50%,283.811371,1.553374,0.583293,2.556624,0.034953,0.139830,-0.007431,0.040936
75%,285.349258,1.919869,0.678760,3.770207,0.092390,0.217083,0.635056,0.444027
max,291.222870,4.129824,0.919062,7.057856,0.320617,0.546873,2.494878,1.528951


In [16]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X
X.describe(include='all')

,0,1,2,3,4,5,6,7
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,-127.600945,-3.228542,-4.213255,-1.148140,-0.226287,-1.157058,0.039470,-0.032361
std,0.449567,1.981839,7.269499,0.505656,9.936366,8.453327,1.089986,1.686238
min,-129.092926,-7.845831,-25.073909,-2.736924,-45.126769,-35.700559,-3.089337,-4.994491
25%,-127.923889,-4.677430,-9.453660,-1.511322,-5.728033,-7.142715,-0.803574,-1.197663
50%,-127.618130,-3.527164,-4.024154,-1.075173,0.975124,-0.948354,0.073658,0.029427
75%,-127.307451,-2.088352,1.018510,-0.765017,6.643311,4.569513,0.836624,1.175047
max,-126.120882,6.587659,13.711545,0.075209,29.166137,28.125048,3.045202,4.258498


In [17]:
regLin=load('../../March2021/Observation_models/LRModel.joblib') 

In [18]:
yLR=regLin.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})

In [19]:
dump(y_predLin_ds.yLR,'../Model_plots/GFDL-CM4-LR-Y.joblib')

['../Model_plots/GFDL-CM4-LR-Y.joblib']